# Test connection to Chat GPT in AI Foundry


### Get the deployment outputs

We are now at the stage where we only need to retrieve the gateway URL and the subscription before we are ready for testing.

In [7]:
azure_openai_endpoint = ! terraform output -raw azure_openai_endpoint
azure_openai_endpoint = azure_openai_endpoint.n
print("👉🏻 Azure OpenAI Endpoint: ", azure_openai_endpoint)

azure_openai_api_key = ! terraform output -raw azure_openai_api_key
azure_openai_api_key = azure_openai_api_key.n
print("👉🏻 Azure OpenAI Key: ", azure_openai_api_key)

azure_openai_deployment_name = ! terraform output -raw azure_openai_deployment_name
azure_openai_deployment_name = azure_openai_deployment_name.n
print("👉🏻 Azure OpenAI Deployment Name: ", azure_openai_deployment_name)

azure_openai_api_version = "2024-10-21"
openai_model_name = "gpt-4o"

👉🏻 Azure OpenAI Endpoint:  https://ai-services-43854-400007.cognitiveservices.azure.com/
👉🏻 Azure OpenAI Key:  2KJ5R0jrfDWG7ooYNs9aKcDebXNHVHkbgLxhDLIhnKgLoLclbczpJQQJ99BDACYeBjFXJ3w3AAAAACOGwBjM
👉🏻 Azure OpenAI Deployment Name:  gpt-4o


<a id='requests'></a>
### 🧪 Test the API using a direct HTTP call
Requests is an elegant and simple HTTP library for Python that will be used here to make raw API requests and inspect the responses. 

You will not see HTTP 429s returned as API Management's `retry` policy will select an available backend. If no backends are viable, an HTTP 503 will be returned.

Tip: Use the [tracing tool](../../tools/tracing.ipynb) to track the behavior of the backend pool.

In [8]:
import json
import requests

url = azure_openai_endpoint + "/openai/deployments/" + azure_openai_deployment_name + "/chat/completions?api-version=" + azure_openai_api_version

messages={"messages":[
    {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]}

response = requests.post(url, headers = {'api-key': azure_openai_api_key}, json = messages)

# Check the response status code and apply formatting
if 200 <= response.status_code < 300:
    status_code_str = '\x1b[1;32m' + str(response.status_code) + " - " + response.reason + '\x1b[0m'  # Bold and green
elif response.status_code >= 400:
    status_code_str = '\x1b[1;31m' + str(response.status_code) + " - " + response.reason + '\x1b[0m'  # Bold and red
else:
    status_code_str = str(response.status_code)  # No formatting
    
# Print the response status with the appropriate formatting
print("Response status:", status_code_str)

if (response.status_code == 200):
    data = json.loads(response.text)
    print("Token usage:", data.get("usage"), "\n")
    print("💬 ", data.get("choices")[0].get("message").get("content"), "\n")
else:
    print(response.text)   

Response status: 200 - OK
Token usage: {'completion_tokens': 57, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens': 30, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'total_tokens': 87} 

💬  Oh, sure! Let me just consult my magical time-telling powers... Oh wait, I forgot — I don't have a clock because I'm stuck in your device. Why don't you try looking at the phone you're using to talk to me? Genius idea, right? You're welcome. 



<a id='sdk'></a>
### 🧪 Test the API using the Azure OpenAI Python SDK

Repeat the same test using the Python SDK to ensure compatibility.

In [9]:
from openai import AzureOpenAI

messages=[
    {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]

client = AzureOpenAI(
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_api_key,
    api_version=azure_openai_api_version
)

response = client.chat.completions.create(model=azure_openai_deployment_name, messages=messages)

print("💬 ", response.choices[0].message.content)

💬  Oh, sure! Let me just consult my imaginary watch that's always conveniently set to "figure it out yourself" o'clock.
